# i18

The i18n library provides you an easier way to manage languages with grammars and lexers.

## Replace a string given a grammar and a replace function


```python
sub(grammar, fn_replace, string)
```

In Lark syntax, where the grammar expects a group symbol, fn_replace is a function that replaces this group symbol; any other symbols remain unchanged. The string adheres to the rules of a Context-Free Grammar (CFG) rather than a regular expression (regex). 

In [17]:
from i18.parser import sub

sub("""
start: expression

expression: atom ("+"|"-"|"*"|"/") expression | atom

atom: SIGNED_NUMBER -> group
    | "(" expression ")"

%import common.SIGNED_NUMBER
%import common.WS
%ignore WS

""", lambda token: str(int(token)%2), "(4+( 5 + 6 ) + 5 )     +3")

'(0+( 1 + 0 ) + 1 )     +1'

## Translate your blade templates

When you have embedded code in HTML, you can translate both without losing any information

In [20]:
from i18.grammars import html_grammar
from i18.parser import sub
from deep_translator import (GoogleTranslator)

translator = GoogleTranslator(source='auto', target='de')

new_text = sub(html_grammar, lambda token: translator.translate(text=token), """
<span class="flex">
     @if($step == 1)
            <span class="text-blue-600">Hello <p>World {{$x->g}} 😋</p></span>
     @else
            <span class="text-green-500">You're welcome</span>
     @endif
</span>
""")
print(new_text)


<span class="flex">
     @if($step == 1)
            <span class="text-blue-600">Hallo<p>Welt {{$x->g}} 😋</p></span>
     @else
            <span class="text-green-500">Gern geschehen</span>
     @endif
</span>



# Match in string based on Grammar

```python
match(grammar, text)
```

In Lark syntax, where the grammar expects a group symbol, the text adheres to the rules of a Context-Free Grammar (CFG)..


In [32]:
from i18.parser import match

match("""
start: expression

expression: atom ("+"|"-"|"*"|"/") expression | atom

atom: SIGNED_NUMBER | "(" expression ")" -> group


%import common.SIGNED_NUMBER
%import common.WS
%ignore WS

""", "4+(3+5)")

[Tree(Token('RULE', 'expression'), [Tree(Token('RULE', 'atom'), [Token('SIGNED_NUMBER', '3')]), Tree(Token('RULE', 'expression'), [Tree(Token('RULE', 'atom'), [Token('SIGNED_NUMBER', '5')])])])]

In [2]:
from i18.parser import match
from i18.grammars import json_grammar, sub_in_grammar

email = r"/[\w_.]+@[\w_.]+\.[\w-]{2,4}/"
# JSON Grammar searches in values by default rather than keys.
grammar = sub_in_grammar(json_grammar, {"GROUP": email})
text = '{"key": ["abdiel@apimarket.mx"], "abdiel@apimarket.mx": "2"}'
match(grammar, text)

[Token('__ANON_1', 'abdiel@apimarket.mx')]

# Lexer

A lexer converts a sequence of characters into a sequence of tokens, but these tokens have a meaning associated with them. This task is quite similar to named-entity recognition (yes, you could use Spacy), however, a classic lexer does the job.

In [4]:
from i18.lexer import Lexer
lexer = Lexer()
[token for token in lexer.tokenize("""{{!!$person->name!!}}""")]

[LexToken(BLADE_ECHO,'{{!!$person->name!!}}',1,0)]

# Apply i18

In [5]:
from i18 import apply_i18

new_text, translations = apply_i18("""
<span class="flex">
     <p>Some tokens doesn't make sense to apply internalization, for instance 123 🔥</p>   
     @if($i18->awesome == false)
            <span class="text-blue-600">Hello {{$person->name}},<p>how are you? 😋</p></span>
     @else
            <span class="text-green-500">You're welcome</span>
     @endif
</span>
""")
print(new_text)


<span class="flex">
     <p>{{__("some_tokens_doesn_t_make_sense_to_apply_internalization_for_instance")}} 123 🔥</p>   
     @if($i18->awesome == false)
            <span class="text-blue-600">{{__("hello")}} {{$person->name}},<p>{{__("how_are_you?")}} 😋</p></span>
     @else
            <span class="text-green-500">{{__("you_re_welcome")}}</span>
     @endif
</span>



In [6]:
translations

{'es': {'some_tokens_doesn_t_make_sense_to_apply_internalization_for_instance': 'Algunos tokens no tienen sentido aplicar la internalización, por ejemplo',
  'hello': 'Hola',
  'how_are_you?': '¿Cómo estás?',
  'you_re_welcome': 'De nada'},
 'en': {'some_tokens_doesn_t_make_sense_to_apply_internalization_for_instance': "Some tokens doesn't make sense to apply internalization, for instance",
  'hello': 'Hello',
  'how_are_you?': 'how are you?',
  'you_re_welcome': "You're welcome"}}

# Tests

In [9]:
from i18.parser import parse, print_pretty
from i18.grammars import html_grammar

assert parse(html_grammar, "<div>1</div>").pretty() == "start\n  element\n    tag\tdiv\n    group\t1\n    tag\tdiv\n"
assert parse(html_grammar, '<div @id.g="1">1</div>').pretty() == 'start\n  element\n    tag\tdiv\n    attribute\n      @\n      id.g\n      ="1"\n    group\t1\n    tag\tdiv\n'
assert parse(html_grammar, "<div @id.g='1'>1</div>").pretty() == "start\n  element\n    tag\tdiv\n    attribute\n      @\n      id.g\n      ='1'\n    group\t1\n    tag\tdiv\n"
assert parse(html_grammar, "<div :id.g.x.{{ $x}}='1'>1</div>").pretty() == "start\n  element\n    tag\tdiv\n    attribute\n      :\n      id.g.x.{{ $x}}\n      ='1'\n    group\t1\n    tag\tdiv\n"
assert parse(html_grammar, """
  <linearGradient id=":R2m96:" x1="11.5" y1="18" x2="36" y2="15.5"
                                                gradientUnits="userSpaceOnUse">
                                    <stop offset=".194" stop-color="#fff"></stop>
                                    <stop offset="1" stop-color="#6692F1"></stop>
                                </linearGradient>

""").pretty() == 'start\n  element\n    tag\tlinearGradient\n    attribute\n      id\n      =":R2m96:"\n    attribute\n      x1\n      ="11.5"\n    attribute\n      y1\n      ="18"\n    attribute\n      x2\n      ="36"\n    attribute\n      y2\n      ="15.5"\n    attribute\n      gradientUnits\n      ="userSpaceOnUse"\n    element\n      tag\tstop\n      attribute\n        offset\n        =".194"\n      attribute\n        stop-color\n        ="#fff"\n      tag\tstop\n    element\n      tag\tstop\n      attribute\n        offset\n        ="1"\n      attribute\n        stop-color\n        ="#6692F1"\n      tag\tstop\n    tag\tlinearGradient\n'
assert parse(html_grammar, """
  @error($parameter->name) <span  class="text-sm text-red-500"> {{$message}}</span> @enderror
""").pretty() == 'start\n  element\n    args\n      expression\n        method_call\n          variable\tparameter\n          name\n    start\n      element\n        tag\tspan\n        attribute\n          class\n          ="text-sm text-red-500"\n        group\t {{$message}}\n        tag\tspan\n'
assert parse(html_grammar, """
<div class="p-16">
    <p x-data="{length: 25}"
       x-init="originalContent = $el.firstElementChild.textContent.trim()">
        <span x-text="originalContent.slice(0, length)">Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.</span>
        <button x-cloak @click="length = undefined" x-show="length">...</button>
    </p>
</div>
""").pretty() == 'start\n  element\n    tag\tdiv\n    attribute\n      class\n      ="p-16"\n    element\n      tag\tp\n      attribute\n        x-data\n        ="{length: 25}"\n      attribute\n        x-init\n        ="originalContent = $el.firstElementChild.textContent.trim()"\n      element\n        tag\tspan\n        attribute\n          x-text\n          ="originalContent.slice(0, length)"\n        group\tLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n        tag\tspan\n      group\t\n        \n      element\n        tag\tbutton\n        attribute\tx-cloak \n        attribute\n          @\n          click\n          ="length = undefined"\n        attribute\n          x-show\n          ="length"\n        group\t...\n        tag\tbutton\n      tag\tp\n    tag\tdiv\n'
assert parse(html_grammar, """
 <div>
   @if (is_null($batch->job))
      <div><button>1</button></div> 
   @elseif ($batch->job->hasPendingJobs())
     
   @elseif ($batch->job->finished())
      <div>3</div>
   @else
      <div>4</div>
   @endif   
 </div>
""").pretty() == 'start\n  element\n    tag\tdiv\n    element\n      args\n        expression\n          function_call\n            is_null\n            args\n              expression\n                method_call\n                  variable\tbatch\n                  job\n      start\n        element\n          tag\tdiv\n          element\n            tag\tbutton\n            group\t1\n            tag\tbutton\n          tag\tdiv\n      args\n        expression\n          method_call\n            variable\tbatch\n            job\n            hasPendingJobs\n      start\n        group\t\n     \n   \n      args\n        expression\n          method_call\n            variable\tbatch\n            job\n            finished\n      start\n        element\n          tag\tdiv\n          group\t3\n          tag\tdiv\n      start\n        element\n          tag\tdiv\n          group\t4\n          tag\tdiv\n    tag\tdiv\n'

In [10]:
print_pretty(html_grammar, """
 <div>
   @if (is_null($batch->job))
      <div><button>1</button></div> 
   @elseif ($batch->job->hasPendingJobs())
     <div>1</div>
   @elseif ($batch->job->finished() <> null)
      <div>3</div>
   @else
      <div>4</div>
   @endif   
 </div>
""")

start
  element
    tag	div
    element
      args
        expression
          function_call
            is_null
            args
              expression
                method_call
                  variable	batch
                  job
      start
        element
          tag	div
          element
            tag	button
            group	1
            tag	button
          tag	div
      args
        expression
          method_call
            variable	batch
            job
            hasPendingJobs
      start
        element
          tag	div
          group	1
          tag	div
      args
        expression
          expression
            method_call
              variable	batch
              job
              finished
          binary_op
          expression
            literal	null
      start
        element
          tag	div
          group	3
          tag	div
      start
        element
          tag	div
          group	4
          tag	div
    tag	div



In [4]:
parse(html_grammar, """
<div class="p-16">
    <p x-data="{length: 25}"
       x-init="originalContent = $el.firstElementChild.textContent.trim()">
        <span x-text="originalContent.slice(0, length)">Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.</span>
        <button x-cloak @click="length = undefined" x-show="length">...</button>
    </p>
</div>
""").pretty()

'start\n  element\n    tag\tdiv\n    attribute\n      class\n      ="p-16"\n    element\n      tag\tp\n      attribute\n        x-data\n        ="{length: 25}"\n      attribute\n        x-init\n        ="originalContent = $el.firstElementChild.textContent.trim()"\n      element\n        tag\tspan\n        attribute\n          x-text\n          ="originalContent.slice(0, length)"\n        group\tLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n        tag\tspan\n      group\t\n        \n      element\n        tag\tbutton\n        attribute\tx-cloak \n        attribute\n          @\n          clic